##Creating Face Mask Detector for Android app with optimization##

**Importing all packages**

In [ ]:
!pip install tensorflow_model_optimization

     |████████████████████████████████| 174kB 8.5MB/s 


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
import tensorflow_model_optimization as tfmot
import tensorflow as tf
from tensorflow.keras.models import load_model

**Getting out data**

In [ ]:
from google.colab import files
files.upload()
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive
IMG_HEIGHT, IMG_WIDTH = 224, 224
image_path = '/gdrive/My Drive/Colab Notebooks/Datafiles/Face Mask Detection/data'
image_generator = ImageDataGenerator(validation_split=0.2, rescale=1./255)
train_data_gen = image_generator.flow_from_directory(directory=image_path,shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='categorical',
                                                     subset='training')
val_data_gen = image_generator.flow_from_directory(directory=image_path, class_mode='categorical',target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                   subset='validation')

Saving no_mask.png to no_mask.png
Saving mask.png to mask.png
Mounted at /gdrive
/gdrive
Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.


**Creating model with quantinization**

In [ ]:
base_model=MobileNetV2(weights='imagenet',include_top=False, input_shape=(224,224,3)) #imports the MobileNetV2 model and discards the last 1000 neuron layer.
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(8,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dropout(0.5)(x)
preds=Dense(2,activation='softmax')(x) #final layer with softmax activation for N classes

model=Model(inputs=base_model.input,outputs=preds) #specify the inputs and outputs)

for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True
model.compile(loss = 'categorical_crossentropy',
              optimizer = tf.keras.optimizers.RMSprop(
    learning_rate=0.0001, rho=0.9, momentum=0.1, epsilon=1e-07, centered=False,
),
              metrics = ['accuracy'])


9412608/9406464 [==============================] - 0s 0us/step


**Training model**

In [ ]:
BATCH_SIZE = 16
epochs = 10
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.0000001)
callbacks = [earlystop,learning_rate_reduction]
# Fit the model on Training data
model.fit(train_data_gen, epochs=epochs, validation_data=val_data_gen, 
          batch_size=BATCH_SIZE, callbacks=callbacks)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
%cd ~
# Save the model.
with open('/content/mask_detector.tflite', 'wb') as f:
  f.write(tflite_model)
model.save('/content/mask_detector.h5')

Epoch 1/10
 19/189 [==>...........................] - ETA: 22:02 - loss: 0.5321 - accuracy: 0.7213

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


189/189 [==============================] - 1852s 10s/step - loss: 0.2584 - accuracy: 0.8867 - val_loss: 0.1857 - val_accuracy: 0.9695
Epoch 2/10
189/189 [==============================] - 35s 183ms/step - loss: 0.0590 - accuracy: 0.9635 - val_loss: 0.0489 - val_accuracy: 0.9940
Epoch 3/10
189/189 [==============================] - 35s 184ms/step - loss: 0.0519 - accuracy: 0.9652 - val_loss: 0.2862 - val_accuracy: 0.9735
Epoch 4/10
189/189 [==============================] - 35s 184ms/step - loss: 0.0508 - accuracy: 0.9654 - val_loss: 0.5732 - val_accuracy: 0.9444

Epoch 00004: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
Epoch 5/10
189/189 [==============================] - 34s 182ms/step - loss: 0.0448 - accuracy: 0.9712 - val_loss: 0.3074 - val_accuracy: 0.9682
Epoch 6/10
189/189 [==============================] - 35s 183ms/step - loss: 0.0431 - accuracy: 0.9648 - val_loss: 0.1324 - val_accuracy: 0.9861

Epoch 00006: ReduceLROnPlateau reducing learning rate to 2.

##Clustering##


In [ ]:
import tensorflow as tf
import numpy as np
import tempfile
import os
import tensorflow_model_optimization as tfmot


def setup_model():
  base_model=MobileNetV2(weights='imagenet',include_top=False, input_shape=(224,224,3)) #imports the MobileNetV2 model and discards the last 1000 neuron layer.
  x=base_model.output
  x=GlobalAveragePooling2D()(x)
  x=Dense(8,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
  x=Dropout(0.5)(x)
  preds=Dense(2,activation='softmax')(x) #final layer with softmax activation for N classes

  model=Model(inputs=base_model.input,outputs=preds)
  for layer in model.layers[:20]:
    layer.trainable=False
  for layer in model.layers[20:]:
    layer.trainable=True
  return model

def train_model(model):
  model.compile(loss = 'categorical_crossentropy',
              optimizer = tf.keras.optimizers.RMSprop(
    learning_rate=0.0001, rho=0.9, momentum=0.1, epsilon=1e-07, centered=False,
),
              metrics = ['accuracy'])
  BATCH_SIZE = 16
  epochs = 10
  from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
  earlystop = EarlyStopping(patience = 10)
  learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.0000001)
  callbacks = [earlystop,learning_rate_reduction]
  model.summary()
  # Fit the model on Training data
  model.fit(train_data_gen, epochs=epochs, validation_data=val_data_gen, 
            batch_size=BATCH_SIZE, callbacks=callbacks)

  return model

def save_model_weights(model):
  _, pretrained_weights = tempfile.mkstemp('.h5')
  model.save_weights(pretrained_weights)
  return pretrained_weights

def setup_pretrained_weights():
  model= setup_model()
  model = train_model(model)
  pretrained_weights = save_model_weights(model)
  return pretrained_weights

def setup_pretrained_model():
  model = setup_model()
  pretrained_weights = setup_pretrained_weights()
  model.load_weights(pretrained_weights)
  return model

def save_model_file(model):
  _, keras_file = tempfile.mkstemp('.h5') 
  model.save(keras_file, include_optimizer=False)
  return keras_file

def get_gzipped_model_size(model):
  # It returns the size of the gzipped model in bytes.
  import os
  import zipfile

  keras_file = save_model_file(model)

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(keras_file)
  return os.path.getsize(zipped_file)

setup_model()
pretrained_weights = setup_pretrained_weights()



Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


189/189 [==============================] - 2633s 14s/step - loss: 0.2771 - accuracy: 0.8512 - val_loss: 1.1248 - val_accuracy: 0.8821
Epoch 2/10
189/189 [==============================] - 36s 190ms/step - loss: 0.0780 - accuracy: 0.9616 - val_loss: 0.0914 - val_accuracy: 0.9841
Epoch 3/10
189/189 [==============================] - 36s 188ms/step - loss: 0.0621 - accuracy: 0.9647 - val_loss: 0.3825 - val_accuracy: 0.9642
Epoch 4/10
189/189 [==============================] - 35s 187ms/step - loss: 0.0489 - accuracy: 0.9659 - val_loss: 0.1008 - val_accuracy: 0.9887
Epoch 5/10
189/189 [==============================] - 36s 189ms/step - loss: 0.0516 - accuracy: 0.9619 - val_loss: 0.2604 - val_accuracy: 0.9788
Epoch 6/10
189/189 [==============================] - 35s 187ms/step - loss: 0.0418 - accuracy: 0.9686 - val_loss: 0.4502 - val_accuracy: 0.9649

Epoch 00006: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
Epoch 7/10
189/189 [==============================] - 35s 18

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# Define the model.
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
batch_size = 16
epochs = 20
base_model = setup_model()
base_model.load_weights(pretrained_weights) # optional but recommended for model accuracy
num_images = 6043 
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.8,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

#model_for_pruning.summary()
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.0000001)
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
  learning_rate_reduction, earlystop,
]

model_for_pruning.fit(train_data_gen, validation_data= val_data_gen,
                  batch_size=batch_size, epochs=epochs,
                  callbacks=callbacks)
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Epoch 1/20
  6/189 [..............................] - ETA: 2:42 - loss: 0.1589 - accuracy: 0.9460WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1894s vs `on_train_batch_end` time: 0.5517s). Check your callbacks.


 12/189 [>.............................] - ETA: 1:41 - loss: 0.1360 - accuracy: 0.9598

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


189/189 [==============================] - 91s 335ms/step - loss: 0.1122 - accuracy: 0.9678 - val_loss: 0.4306 - val_accuracy: 0.9219
Epoch 2/20
189/189 [==============================] - 60s 316ms/step - loss: 0.1051 - accuracy: 0.9711 - val_loss: 0.2318 - val_accuracy: 0.9656
Epoch 3/20
189/189 [==============================] - 60s 317ms/step - loss: 0.1018 - accuracy: 0.9682 - val_loss: 0.3012 - val_accuracy: 0.9338
Epoch 4/20
189/189 [==============================] - 60s 317ms/step - loss: 0.0978 - accuracy: 0.9710 - val_loss: 0.5193 - val_accuracy: 0.9325

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 5/20
189/189 [==============================] - 59s 314ms/step - loss: 0.1035 - accuracy: 0.9664 - val_loss: 0.4320 - val_accuracy: 0.9417
Epoch 6/20
189/189 [==============================] - 60s 316ms/step - loss: 0.1025 - accuracy: 0.9666 - val_loss: 0.3323 - val_accuracy: 0.9517

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.

"_, pruned_keras_file = tempfile.mkstemp('.h5')\ntf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)\nprint('Saved pruned Keras model to:', pruned_keras_file)\nconverter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)\npruned_tflite_model = converter.convert()\n\n_, pruned_tflite_file = tempfile.mkstemp('.tflite')\n\n%cd ~\n# Save the model.\nwith open('/content/mask_detector_after_pruning', 'wb') as f:\n  f.write(tflite_model)\n\nprint('Saved pruned TFLite model to:', pruned_tflite_file)"

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

_, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')
%cd ~
# Save the model.
with open('/content/mask_detector.tflite', 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)

with open(quantized_and_pruned_tflite_file, 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

#print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
#print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))

INFO:tensorflow:Assets written to: /tmp/tmpz2v2tpuv/assets


INFO:tensorflow:Assets written to: /tmp/tmpz2v2tpuv/assets


/root
Saved quantized and pruned TFLite model to: /tmp/tmpu1u9vdzr.tflite


In [ ]:
import tensorflow as tf

import numpy as np
import cv2


interpreter = tf.lite.Interpreter(model_path="/content/mask_detector.tflite")


img_no = cv2.imread("/content/no_mask.png")[:, :, ::-1]
img_no = cv2.resize(img_no, (224, 224))
img_no = np.array([img_no]) / 255


img_with = cv2.imread("/content/mask.png")[:, :, ::-1]
img_with = cv2.resize(img_with, (224, 224))
img_with = np.array([img_with]) / 255


def run(img):
    interpreter.allocate_tensors()
    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    # Test model on random input data.
    input_data = np.array(img, dtype=np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])

    return output_data

output_data = run(img_no)
print(f"no_mask.png: mask: {output_data[0,0]:.05f} | no mask: {output_data[0,1]:.05f}")

output_data = run(img_with)
print(f"mask.png: mask: {output_data[0,0]:.05f} | no mask: {output_data[0,1]:.05f}")


no_mask.png: mask: 0.00000 | no mask: 1.00000
mask.png: mask: 0.99911 | no mask: 0.00089
